In [1]:
import tensorflow as tf
import utils
import os

from dataset import get_datasets
from stormer import Stormer

2024-05-08 05:18:10.384133: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 05:18:10.451970: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 05:18:11.363185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/zuherj/miniconda3/envs/kws/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/e

In [2]:
# count all the files inside "data/speech_commands_v0.02" directory
cnt = 0
for root, dirs, files in os.walk("data/speech-commands-v2"):
    for file in files:
        if file.endswith(".wav"):
            cnt += 1

print(cnt)

105835


In [3]:
# An example to how to initailize the parameters in case you don't have no model

version = 2
num_classes = 37 if version == 2 else 31 #### should change
dataset_type = "mel"
frame_length = 256
frame_step = 128
mel_bands=40
num_coefficients=13
max_shift_in_ms=100

learning_rate = 0.001
weight_decay = 0.005
batch_size = 64
num_epochs = 10000  # For real training, use num_epochs=100. 10 is a test value

num_heads = 4
num_repeats = 2
num_state_cells = [10, 10]
input_seq_size = 31
projection_dim = 32
inner_ff_dim = 2 * projection_dim
dropout = 0.1
probability_of_noise = 0.8

hps = {
    "version": version,
    "num_classes": num_classes,
    "dataset_type": dataset_type,
    "frame_length": frame_length,
    "frame_step": frame_step,
    "mel_bands": mel_bands,
    "num_coefficients": num_coefficients,
    "max_shift_in_ms": max_shift_in_ms,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "num_heads": num_heads,
    "num_repeats": num_repeats,
    "num_state_cells": num_state_cells,
    "input_seq_size": input_seq_size,
    "projection_dim": projection_dim,
    "inner_ff_dim": inner_ff_dim,
    "dropout": dropout,
    "probability_of_noise": probability_of_noise,
}
model_name = utils.get_model_name(**hps)

utils.save_hps(model_name, hps)
model_path = utils.get_model_path(model_name)

In [4]:
stormer = Stormer(
    **hps
)

2024-05-08 05:18:15.218342: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43626 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:27:00.0, compute capability: 8.9


In [5]:
## load the datasets
train, valid, test = get_datasets(
    **hps
)

# get train size using cardinality
train_size = train.cardinality().numpy()

Loading dataset version 2
Dataset loaded


2024-05-08 05:18:18.814327: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-05-08 05:18:18.820452: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [6]:
train_size

1489

In [7]:
results_filename = f'data/results/results_r{num_repeats}_h{num_heads}_dm{projection_dim}_dataset={dataset_type}.csv'

metrics=["accuracy"]

stormer.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate),
    loss="categorical_crossentropy",
    metrics=metrics,
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

state_transformer_history = stormer.fit(
    train,
    validation_data=valid,
    epochs=num_epochs,
    callbacks=[
        model_checkpoint_callback,
        utils.MetricsLogger(
            results_filename,
        )
    ],
)

Epoch 1/10000
